In [1]:
import psycopg2
import random
import requests
import numpy as np
from numpy.random import choice

import time

In [2]:
sectors = ['Private', 'Hotel', 'Gastronomy', 'Public']

s_d_visitors_lb = 1000
s_d_visitors_ub = 10000

s_i_visitors_lb = 500
s_i_visitors_ub = 5000

s_d_visitors_lb2 = 250
s_d_visitors_ub2 = 1000

s_i_visitors_lb2 = 50
s_i_visitors_ub2 = 300

t_i_visitors_lb = 300
t_i_visitors_ub = 1000

i_i_visitors_lb = 300
i_i_visitors_ub = 1000

w_visitors_lb = 50
w_visitors_ub = 300

In [3]:
def save_visitor(visitor, location_id, language, telephone):
    name = visitor['name']
    surname = visitor['surname']
    gender = visitor['gender']
    
    customer_number = f'\'{"".join([str(random.randint(0,9)) for _ in range(10)])}\'' if random.random() < 0.3 else 'NULL'
    email = f'\'{name.lower()}.{surname.lower()}@email.com\'' if customer_number !='NULL' or random.random() < 0.7 else 'NULL'
    telephone = f'\'{telephone}\'' if random.random() < 0.5 else 'NULL'
    
    sector = choice(sectors, p=[0.4, 0.3, 0.2, 0.1])
    
    return f'(\'{name} {surname}\', {telephone}, {email}, \'{gender}\', \'{language}\', {location_id}, \'{sector}\', {customer_number})'
    


def create_s_d_visitors(cities, lb, ub):
    values = []
    num_visitors_de = random.randint(lb,ub)
    
    while (num_visitors_de > 0):
        if (num_visitors_de > 500):
            num = 500
        else:
            num = num_visitors_de
            
        print(num,end=', ')
        
        res = requests.get(f'http://uinames.com/api/?region=austria&amount={num}')
        visitors_s_d = res.json()

        for visitor in visitors_s_d:
            location_id = cities[random.randrange(len(cities))][0]
            telephone = f'3{"".join([str(random.randint(0,9)) for _ in range(2)])} {"".join([str(random.randint(0,9)) for _ in range(7)])}'
            values.append(save_visitor(visitor, location_id, 'german', telephone))
        
        num_visitors_de -= 500
    
    print()
    return values
    
    
def create_s_i_visitors(cities, lb, ub):
    values = []
    num_visitors_it = random.randint(lb,ub)
    
    while (num_visitors_it > 0):
        if (num_visitors_it > 500):
            num = 500
        else:
            num = num_visitors_it
           
        print(num,end=', ')
        res = requests.get(f'http://uinames.com/api/?region=italy&amount={num}')
        visitors_s_i = res.json()

        for visitor in visitors_s_i:
            location_id = cities[random.randrange(len(cities))][0]
            telephone = f'3{"".join([str(random.randint(0,9)) for _ in range(2)])} {"".join([str(random.randint(0,9)) for _ in range(7)])}'
            values.append(save_visitor(visitor, location_id, 'italian', telephone))
            
        num_visitors_it -= 500
    
    print()
    return values

    
def create_t_i_visitors(lb, ub):
    values = []
    cur.execute('Select location_id FROM warehouse.location where city is null and province = \'Trento\'')
    provinces = cur.fetchall()

    num_visitors_it = random.randint(lb, ub)
    
    while (num_visitors_it > 0):
        if (num_visitors_it > 500):
            num = 500
        else:
            num = num_visitors_it
        print(num,end=', ')

        res = requests.get(f'http://uinames.com/api/?region=italy&amount={num}')
        visitors_t_i = res.json()

        for visitor in visitors_t_i:
            location_id = provinces[random.randrange(len(provinces))][0]
            telephone = f'3{"".join([str(random.randint(0,9)) for _ in range(2)])} {"".join([str(random.randint(0,9)) for _ in range(7)])}'
            values.append(save_visitor(visitor, location_id, 'italian', telephone))
            
        num_visitors_it -= 500
        
    print()
    return values

def create_i_i_visitors(lb, ub):
    values = []
    cur.execute('Select location_id FROM warehouse.location where city is null and province is not null and province != \'Trento\'')
    provinces = [p[0] for p in cur.fetchall()]
    p_province = [random.randint(5,20) for _ in range(len(provinces))]
    p_province = p_province / np.sum(p_province)

    num_visitors_it = random.randint(lb, ub)
    
    while (num_visitors_it > 0):
        if (num_visitors_it > 500):
            num = 500
        else:
            num = num_visitors_it

        print(num,end=', ')
        res = requests.get(f'http://uinames.com/api/?region=italy&amount={num}')
        visitors_s_i = res.json()

        for visitor in visitors_s_i:
            location_id = choice(provinces, p=p_province)
            telephone = f'3{"".join([str(random.randint(0,9)) for _ in range(2)])} {"".join([str(random.randint(0,9)) for _ in range(7)])}'
            values.append(save_visitor(visitor, location_id, 'italian', telephone))
            
        num_visitors_it -= 500
    
    print()
    return values

def create_w_visitors(lb, ub):
    values = []
    cur.execute('Select location_id, country FROM warehouse.location where province is null')
    countries_res = cur.fetchall()
    countries = [c[0] for c in countries_res]
    p_country = [0] * len(countries)
    for i in range(len(countries)):
        country = countries_res[i][1]
        if((country == 'Österreich') | (country == 'Schweiz')):
            p_country[i] = random.randint(20,40)
        elif((country == 'Deutschland') | (country == 'Frankreich')):
            p_country[i] = random.randint(10,15)
        else:
            p_country[i] = random.randint(0,5)
    p_country = p_country / np.sum(p_country)
        
    num_visitors_world = random.randint(lb,ub)
    
    while (num_visitors_world > 0):
        if (num_visitors_world > 500):
            num = 500
        else:
            num = num_visitors_world
        
        print(num,end=', ')

        res = requests.get(f'http://uinames.com/api/?amount={num}')
        visitors_w = res.json()

        for visitor in visitors_w:
            location_id = countries[random.randrange(len(countries))]
            telephone = f'{"".join([str(random.randint(0,9)) for _ in range(3)])} {"".join([str(random.randint(0,9)) for _ in range(7)])}'
            values.append(save_visitor(visitor, location_id, 'english', telephone))
            
        num_visitors_world -= 500
    print()    
    return values

In [4]:
def create_showroom_visits(showroom, visitors, sales_reps, departments, holidays, non_holidays, visitor_types):
    random.shuffle(departments)
    p_sales_rep = np.linspace(20,5,len(sales_reps))
    p_sales_rep = p_sales_rep / np.sum(p_sales_rep)
    sales_rep_positions = np.arange(len(sales_reps))
    
    sales_rep_deps = [choice(departments, size = random.randint(1,3)) for _ in sales_rep_positions]
    
    p_dep = [np.linspace(21,1,len(d)) for d in sales_rep_deps]
    p_dep = [d / np.sum(d) for d in p_dep]
    
    p_vis = [0.4,0.25,0.20,0.1,0.025,0.015,0.01]
    n_vis = [1,2,3,4,5,6,7]
    
    p_is_hol = random.randrange(1, 3) / 10.0
    p_hol = np.arange(1,len(holidays[0]) + 1)
    p_hol = p_hol / np.sum(p_hol)
    random.shuffle(p_hol)
    
    p_day = [4,6,8,12,20,24,20]
    p_day = [random.randint(lb-2, lb+3) for lb in p_day]
    p_day = p_day / np.sum(p_day)
    
    p_month = np.array([20,16,12,8,6,4,4,6,8,12,16,20])
    p_month = [random.randint(lb-2, lb+4) for lb in p_month]
    p_month = p_month / np.sum(p_month)
    
    p_year = [random.randint(lb+4, lb+14) for lb in np.arange(1,len(non_holidays) + 1)]
    p_year = p_year / np.sum(p_year)
    
    values = []
    
    for visitor in visitors:
        n_visits = choice(n_vis, p=p_vis)       
        for i in range(n_visits):
            sales_rep_pos = choice(sales_rep_positions, p=p_sales_rep)
            sales_rep_id = sales_reps[sales_rep_pos]
            department_id = choice(sales_rep_deps[sales_rep_pos],p=p_dep[sales_rep_pos])
            if(random.random() < p_is_hol):
                year = choice(np.arange(0,len(holidays)), p=p_year)
                date_id = choice(holidays[year], p=p_hol)
            else:
                year = choice(np.arange(0,len(non_holidays)), p=p_year)
                selected_year = non_holidays[year]
                month = choice(np.arange(0,len(selected_year)), p=p_month)
                selected_month = selected_year[month]
                day = choice(np.arange(0,len(selected_month)), p=p_day)
                date_id = choice(selected_month[day])
            
            visitor_type = random.choice(visitor_types)
            visitor_type_id = visitor_type[0]
            if(visitor_type[1] == 'Single'):
                num_visitors = 1
            elif(visitor_type[1] == 'Pair'):
                num_visitors = 2
            else:
                num_visitors = random.randint(3,10)
            
            duration = random.randint(5,200)
            
            values.append(f'({visitor}, {sales_rep_id}, {showroom}, {department_id}, {date_id}, {visitor_type_id}, {duration}, {num_visitors})')
    
    query_args = ','.join(values)
    cur.execute(f'insert into warehouse.showroom_visit (visitor_id, sales_rep_id, showroom_id, department_id, date_id, visitor_type_id, duration, number_of_visitors) \
    values {query_args}')

In [5]:
conn = psycopg2.connect(dbname='admt_g02', user='admt_g02', password='xi6aeSoo', host='alcor.inf.unibz.it', port=5433)
cur = conn.cursor()

In [6]:
cur.execute('delete from  warehouse.showroom_visit')
cur.execute('Delete from warehouse.visitor')
cur.execute('Delete from warehouse.order')

In [7]:
start = time.time()
cur.execute('select s.showroom_id, l.district from warehouse.showroom as s join warehouse.location as l on s.location_id = l.location_id')
showrooms = cur.fetchall()

cur.execute('select sales_rep_id from warehouse.sales_representative')
sales_reps = [s[0] for s in cur.fetchall()]
random.shuffle(sales_reps)
n_sales_rep_bz = len(sales_reps) // 4
n_sales_rep = (len(sales_reps) - n_sales_rep_bz) // (len(showrooms) - 1)
n_extra_sales_rep = (len(sales_reps) - n_sales_rep_bz) % (len(showrooms) - 1)

cur.execute('select department_id from warehouse.department')
departments = [d[0] for d in cur.fetchall()]

cur.execute('select * from warehouse.visitor_type')
visitor_types = cur.fetchall()

cur.execute('select distinct year_actual from warehouse.date order by year_actual')
years = [y[0] for y in cur.fetchall()]

holidays = []
for year in years:
    cur.execute('select date_id from warehouse.date where date.holiday and year_actual = %d' % year)
    holidays.append([d[0] for d in cur.fetchall()])

non_holidays = []
for year in years:
    non_holiday_years = []
    for m in range(1,13):
        non_holiday_months = []
        for d in range(1,8):
            cur.execute('select date_id from warehouse.date where not date.holiday and day_of_week = %d \
            and month_actual = %d and year_actual = %d' % (d,m,year))
            non_holiday_months.append([d[0] for d in cur.fetchall()])
        non_holiday_years.append(non_holiday_months)
    non_holidays.append(non_holiday_years)

print('Inital setup:', np.round(time.time() - start, 2), '\n')

all_visitor_ids = [[]] * len(showrooms)
all_sales_rep_ids = [[]] * len(showrooms)

curr_sales_rep = 0
for i, showroom in enumerate(showrooms):
    start = time.time()
    print(showroom[1])
    cur.execute('Select location_id FROM warehouse.location where district = \'%s\'' % showroom[1])
    cities = cur.fetchall()
    
    factor = 2 if showroom[1] == 'Bozen' else 1
    visitors = []
    
    s_d_visitors = create_s_d_visitors(cities, s_d_visitors_lb*factor, s_d_visitors_ub*factor) 
    visitors.extend(s_d_visitors)
    if showroom[1] == 'Bozen':
        s_i_visitors = create_s_i_visitors(cities, s_i_visitors_lb*(factor*10), s_i_visitors_ub*(factor**2))
    else:
        s_i_visitors = create_s_i_visitors(cities, s_i_visitors_lb, s_i_visitors_ub)
    visitors.extend(s_i_visitors)
    
    cur.execute('Select location_id FROM warehouse.location where district != \'%s\'' % showroom[1])
    cities2 = cur.fetchall()
    
    s_d_visitors2 = create_s_d_visitors(cities2, s_d_visitors_lb2*factor, s_d_visitors_ub2*factor)
    visitors.extend(s_d_visitors2)
    s_i_visitors2 = create_s_i_visitors(cities2, s_i_visitors_lb2*factor, s_i_visitors_ub2*factor)
    visitors.extend(s_i_visitors2)
    
    t_i_visitor = create_t_i_visitors(t_i_visitors_lb*(factor**2), t_i_visitors_ub*(factor**2))
    visitors.extend(t_i_visitor)
    i_i_visitor = create_i_i_visitors(i_i_visitors_lb*(factor**2), i_i_visitors_ub*(factor**2))
    visitors.extend(i_i_visitor)
    w_visitor = create_w_visitors(w_visitors_lb*(factor**3), w_visitors_ub*(factor**3))
    visitors.extend(w_visitor)
    
    query_args = ','.join(visitors)
    
    print('Formed query:', np.round(time.time() - start, 2))
    start = time.time()
    
    cur.execute(f'Insert into warehouse.visitor (visitor_name, visitor_telephone, visitor_email, \
    visitor_gender, visitor_language, location_id, visitor_sector, customer_number) values {query_args}  returning visitor_id')
    
    visitor_ids = [v[0] for v in cur.fetchall()]
    all_visitor_ids[i] = np.array(visitor_ids)
    print('Fetched visitors', np.round(time.time() - start, 2))
    
    next_sales_rep = curr_sales_rep + (n_sales_rep_bz if showroom[1] == 'Bozen' else n_sales_rep)
    if n_extra_sales_rep > 0:
        next_sales_rep += 1
        n_extra_sales_rep -= 1
        
    curr_sales_reps = sales_reps[curr_sales_rep:next_sales_rep]
    all_sales_rep_ids[i] = curr_sales_reps
        
    start = time.time()
    create_showroom_visits(showroom[0], visitor_ids, curr_sales_reps, departments, holidays, non_holidays, visitor_types)
    curr_sales_rep = next_sales_rep
    print('Created visit', np.round(time.time() - start, 2))  
    
    print('\n')
    

Inital setup: 2.86 

Bozen
500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 358, 
500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 136, 
500, 355, 
500, 92, 
500, 500, 500, 500, 500, 500, 200, 
500, 500, 500, 500, 165, 
494, 
Formed query: 69.08
Fetched visitors 1.69
Created visit 25.19


Vinschgau
500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 191, 
500, 500, 500, 321, 
500, 202, 
209, 
500, 137, 
500, 494, 
92, 
Formed query: 25.6
Fetched visitors 0.53
Created visit 8.09


Salten Schlern
500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 110, 
500, 466, 
500, 190, 
241, 
500, 214, 
500, 102, 
212, 
Formed query: 22.84
Fetched visitors 0.47
Created visit 7.58


Eisacktal
500, 500, 500, 500, 500, 500, 

In [8]:
for i, showroom in enumerate(showrooms):
    other_visitors = np.hstack(all_visitor_ids[:i] + all_visitor_ids[i+1:])
    selected_visitors = choice(other_visitors, len(other_visitors) // random.randint(15, 30))
    
    start = time.time()
    create_showroom_visits(showroom[0], selected_visitors, all_sales_rep_ids[i], departments, holidays, non_holidays, visitor_types)
    print('Created visit', np.round(time.time() - start, 2)) 
    
    

Created visit 3.46
Created visit 2.65
Created visit 4.34
Created visit 2.39
Created visit 4.43
Created visit 1.55
Created visit 4.39
Created visit 3.11


In [9]:
for showroom in showrooms:
    start = time.time()
    cur.execute('select showroom_visit_id FROM warehouse.showroom_visit where showroom_id = %d' %showroom[0])
    visits = [v[0] for v in cur.fetchall()]
    
    selected_visits = choice(visits, len(visits) // random.randint(20, 50))
    values = []
    
    print('Selected visits', np.round(time.time() - start, 2), len(selected_visits)) 
    
    start = time.time()
    for visit in selected_visits:
        order_number = f'\'{"".join([str(random.randint(0,9)) for _ in range(10)])}\''
        price = random.randint(5,65000) + (random.randint(1,99) / 100.0)
        values.append(f'({order_number}, {price})')
    
    query_args = ','.join(values)
    cur.execute(f'insert into warehouse.order (order_number, order_total_price) values {query_args} returning order_id')
    order_ids = [o[0] for o in cur.fetchall()]
    
    updates = []
    for i, visit in enumerate(selected_visits):
        updates.append(f'update warehouse.showroom_visit set order_id = {order_ids[i]} where showroom_visit_id = {visit}')
        
    update_queries = '; '.join(updates)
    cur.execute(update_queries)
        
    print('Updated visit', np.round(time.time() - start, 2)) 
    

Selected visits 0.13 2094
Updated visit 0.48
Selected visits 0.1 815
Updated visit 0.2
Selected visits 0.09 1035
Updated visit 0.22
Selected visits 0.07 1236
Updated visit 0.25
Selected visits 0.09 659
Updated visit 0.6
Selected visits 0.08 715
Updated visit 0.15
Selected visits 0.07 2069
Updated visit 0.42
Selected visits 0.1 1252
Updated visit 0.26


In [10]:
conn.commit()
cur.close()
conn.close()